In [1]:
import time
import os
import cv2
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bs4 import BeautifulSoup
import string,re
import gensim

from nltk import sent_tokenize
from gensim.utils import simple_preprocess

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential,layers
from tensorflow.keras.optimizers import Adam
import kerastuner as kt
from keras.utils import image_dataset_from_directory,plot_model,pad_sequences
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import * 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import *
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score


2024-11-25 00:56:14.832674: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 00:56:14.839826: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-25 00:56:14.957497: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-25 00:56:15.017769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-25 00:56:15.125540: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [8]:
from preprocess import generate_training_sequences,SEQUENCE_LENGTH

import json

json_data = json.load(open('mapping.json','r'))

OUTPUT_UNITS = len(json_data)
LOSS = 'sparse_categorical_crossentropy'
LEARNING_RATE = 0.001
NUM_UNITS = [128,64,16]
EPOCHS = 20
BATCH_SIZE = 64 

In [9]:
def train(output_units = OUTPUT_UNITS,num_units = NUM_UNITS,loss = LOSS,learning_rate= LEARNING_RATE):

    # generate the training sequences
    inputs,targets = generate_training_sequences(SEQUENCE_LENGTH)


    # build the model
    model = build_model(output_units,num_units,loss,learning_rate)
    
    # train the model
    model.fit(inputs,targets,epochs = EPOCHS,batch_size= BATCH_SIZE)

    # output the model
    
    model.save("my_model.keras")

In [12]:
def build_model(output_units,num_units,loss,learning_rate):
    model = Sequential()
    model.add(Input(shape = (None,output_units)))
    model.add(LSTM(num_units[0],return_sequences=True))
    
    model.add(Dropout(0.2))
    model.add(LSTM(num_units[1],return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(num_units[2]))

    model.add(Dense(output_units,activation='softmax'))
    
    model.compile(loss=loss,optimizer = Adam(learning_rate=learning_rate),metrics = ['accuracy'])
    
    model.summary()

    return model

In [13]:
train()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, None, 128)      │        86,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, None, 64)       │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 16)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40)             │           680 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 141,800 (553.91 KB)

 Trainable params: 141,800 (553.91 KB)

 Non-trainable params: 0 (0.00 B)

: 

: 

: 